In [1]:
import pandas as pd

Загрузим табличку с нашими данными про длительность поездок такси! 

In [10]:
df = pd.read_csv('taxi_dataset_with_target.csv', index_col=0).drop(columns=['pickup_datetime', 'trip_duration'], axis=1)

df.head()

,vendor_id,passenger_count,store_and_fwd_flag,distance_km
id,,,,
id2875421,1,930.399753,0,1.500479
id2377394,0,930.399753,0,1.807119
id3858529,1,930.399753,0,6.392080
id3504673,1,930.399753,0,1.487155
id2181028,1,930.399753,0,1.189925


In [27]:
# df.to_csv('X.csv', index=False)
X = pd.read_csv('X.csv')
X.head()

,vendor_id,passenger_count,store_and_fwd_flag,distance_km
0,1,930.399753,0,1.500479
1,0,930.399753,0,1.807119
2,1,930.399753,0,6.392080
3,1,930.399753,0,1.487155
4,1,930.399753,0,1.189925


In [18]:
#df['trip_duration'].to_csv('Y.csv', index=False)
Y = pd.read_csv('Y.csv')
Y.head()

,trip_duration
0,455.0
1,663.0
2,2124.0
3,429.0
4,435.0


Попробуем построить модель Линейной Регрессии!

В качестве Выборки используйте таргетную колонку (*trip_duration*) и все признаки, кроме времени начала заказа (*pickup_datetime*)

Для начала воспользуемся "коробочным решением". Для этого создайте переменную model, положите в нее класс **LinearRegression** из модуля **linear_model** библиотеки **sklearn**.

Далее воспользуйтесь методом **fit**, положите в него через запятую: массив **X**, состоящий из признаков объектов (можно pandas dataframe'ом, а можно numpy-массивом), и массив **Y** с таргетами.

In [14]:
from sklearn.linear_model import LinearRegression

LinearRegression()

Чтобы посмотреть на значения полученных коэффициентов модели, следует обратиться к атрибуту **coef_** класса линейной регрессии. Для просмотра свободного веса, к атрибуту **intercept_** 

In [67]:
model=LinearRegression()
model.fit(X, Y)

for column, coef in zip(X.columns, model.coef_):
    print(column, coef)
    
print(model.intercept_)

### можно было так : 
### model.coef_.tolist(), model.intercept_

vendor_id [198.46313674   0.29631295  56.46912165 115.27353763   0.        ]
[171.65658146]


In [49]:
print(f'{np.round(coef, 3)} {np.round(model.intercept_,3)}')

[198.463   0.296  56.469 115.274] [171.657]


In [1]:
### Полученные коэффициенты будут являться ответом на задание 5 в ЛМС.
### Перечислите коэффициенты через запятую и пробел, округлив до 3 знаков после точки.
### Используйте точку в качестве десятичного разделителя.

Теперь реализуйте функцию LinearRegressionByMatrix, которая будет принимать на вход 3 параметра:

Матрицу объект-признак **(X)**, вектор с ответами **(Y)**, булевый параметр **fit_intercept**, цель которого - добавить константный признак (состоящий из единичек), если True, и ничего не делать, если False.

Функция должна вернуть одномерный np.array объект с оцененными **$\beta_1, ..., \beta_n, \beta_0$**

Реализуйте решение через аналитическое решение в матрицах из лекции и практики:

$$
\beta^* = (X^T \cdot X)^{-1} \cdot X^T \cdot Y
$$

In [61]:
"""
:param X: матрица объектов
:param Y: вектор (матрица с 1 столбцом) ответов
:param fit_intercept: добавить ли константную переменную в данные?

:return: одномерный numpy-array с полученными коэффициентами beta
"""

import numpy as np
### Your code is here

def LinearRegressionByMatrix(X: np.array, Y: np.array, fit_intercept: bool = True):
    if fit_intercept:
        X['constant'] = 1
    xxt = np.dot(X.T,X)
    xxt_inv=np.linalg.inv(xxt)
    xxt_inv_xxt = np.dot(xxt_inv, X.T)
    B = np.dot(xxt_inv_xxt, Y)
    B_final = np.around(B, decimals = 3)
    return B_final

In [63]:
X_train = pd.read_csv('X.csv')
Y_train = pd.read_csv('Y.csv')

LinearRegressionByMatrix(X_train, Y_train)

[[198.463]
 [  0.296]
 [ 56.469]
 [115.274]
 [171.657]]


In [ ]:
### можно было и так

def LinearRegressionByMatrix(X, Y, fit_intercept=True):
    
    X = np.c_[ X, np.ones(X.shape[0]) ]
    
    return np.dot(np.dot(np.linalg.inv(np.dot(X.T, X)), X.T), Y)

X_train = df.drop(['trip_duration', 'pickup_datetime'], axis=1).values
Y_train = df['trip_duration'].values

LinearRegressionByMatrix(X_train, Y_train)

In [5]:
### Полученные коэффициенты будут являться ответом на задание 6 в ЛМС.
### Перечислите коэффициенты через запятую и пробел, округлив в большую сторону до 3 знаков после точки.
### Используйте точку в качестве десятичного разделителя.

Получились ли коэффициенты такими же, как и в "коробочном" варианте?

Имея коэффициенты модели, можно восстановить предсказания для каждого объекта!

Сделайте это через операцию матричного произведения матрицы **X** и полученных коэффициентов (как результат работы *LinearRegressionByMatrix*)

In [65]:
### Your code is here
np.dot(X,LinearRegressionByMatrix(X_train, Y_train))

array([[ 818.48453708],
       [ 655.36921008],
       [1382.35898601],
       ...,
       [1548.45408035],
       [ 573.14042938],
       [ 577.94358364]])

In [68]:
model.predict(X)

array([[ 818.7747282 ],
       [ 655.65912268],
       [1382.6469154 ],
       ...,
       [1548.74134353],
       [ 573.4306718 ],
       [ 578.2338068 ]])

In [ ]:
### можно было так

betas = LinearRegressionByMatrix(X_train, Y_train)

np.dot(np.c_[ X_train, np.ones(X_train.shape[0]) ], betas)